In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import datetime
import re

In [2]:
text1 = pd.read_csv('newdata/fraudtext_1.csv',encoding='utf-8')
text2 = pd.read_csv('newdata/fraudtext_2.csv',encoding='utf-8')
text3 = pd.read_csv('newdata/fraudtext_3.csv',encoding='utf-8')
total_data = pd.concat([text1,text2,text3]).sort_values(by=['textid']).reset_index(drop=True)
print(len(total_data))

27825


In [3]:
drop_dup_data = total_data.drop_duplicates(subset=['create_memo'],keep='first').reset_index(drop=True)
print(len(drop_dup_data))

27821


In [4]:
total_data.to_csv('newdata/123.csv',encoding='utf-8',index=False)

In [ ]:
total_data = pd.read_csv('data_fraud_report_ant.csv',encoding='utf-8',sep='\t')
print(len(total_data))

In [8]:
drop_dup_data = total_data.drop_duplicates(subset=['create_memo'],keep='first').reset_index(drop=True)
fraud_data = drop_dup_data[drop_dup_data['is_fraud']==1].reset_index(drop=True)
fraud_data

id  is_fraud  finish_first_type  \
0            15       1.0               11.0   
1            30       1.0               11.0   
2            43       1.0               11.0   
3            56       1.0               11.0   
4            60       1.0               11.0   
5            64       1.0               11.0   
6           119       1.0               11.0   
7           121       1.0               11.0   
8           158       1.0               11.0   
9           179       1.0               11.0   
10          222       1.0               11.0   
11          303       1.0               11.0   
12          325       1.0               11.0   
13          346       1.0               11.0   
14          406       1.0               11.0   
15          422       1.0               11.0   
16          435       1.0               11.0   
17          470       1.0               11.0   
18          479       1.0               11.0   
19          490       1.0               11.0   
20          520       1.0               11.0   
21          542       1.0               11.0   
22          578       1.0               11.0   
23          641       1.0               11.0   
24          668       1.0               11.0   
25          765       1.0               11.0   
26          826       1.0               11.0   
27          831       1.0               11.0   
28          832       1.0               11.0   
29          945       1.0               11.0   
...         ...       ...                ...   
100747  1150120       1.0               11.0   
100748  1150123       1.0               11.0   
100749  1150130       1.0               11.0   
100750  1150141       1.0               11.0   
100751  1150153       1.0               11.0   
100752  1150159       1.0               11.0   
100753  1150160       1.0               11.0   
100754  1150166       1.0               11.0   
100755  1150168       1.0               11.0   
100756  1150170       1.0               11.0   
100757  1150171       1.0               11.0   
100758  1150172       1.0               11.0   
100759  1150189       1.0               11.0   
100760  1150195       1.0               11.0   
100761  1150200       1.0               11.0   
100762  1150201       1.0               11.0   
100763  1150207       1.0               11.0   
100764  1150215       1.0               11.0   
100765  1150227       1.0               11.0   
100766  1150228       1.0               11.0   
100767  1150230       1.0               11.0   
100768  1150234       1.0               11.0   
100769  1150235       1.0               11.0   
100770  1150239       1.0               11.0   
100771  1150249       1.0               11.0   
100772  1150252       1.0               11.0   
100773  1150258       1.0               11.0   
100774  1150266       1.0               11.0   
100775  1150288       1.0               11.0   
100776  1150292       1.0               11.0   

                                              create_memo  task_source  \
0                                 开始说借后面没就没上线了，现在话都不回一个，唉         22.0   
1       他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0   
2                                骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0   
3                       他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0   
4       他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0   
5                         这是交的主持会费，交完后我就后悔了叫她返会费，就直接拉黑我了。         22.0   
6                           他以玩游戏充值送分为由，骗了我五千元，结果人都联系不上了。         22.0   
7       盗用我朋友的QQ号像我骗取4000元钱，一次是通过微信扫描二维码转账给对方1200，一次是通...         22.0   
8       他首先让我找到一个店铺，收藏后付款，然后让我下载了一个代付软件，说让我不要付款，但是还是付了...         22.0   
9       之前玩游戏认识的一个朋友他拉我进群，然后昨天玩游戏群主说帮我代打超时空然后我就把账号给他了，...         22.0   
10      骗子是广西人士真名叫刘衍林，专门以发布人家二手车的图片或者人家私家车的图片，来冒充自己是二手...         22.0   
11              对方可以提升信用卡额度，提高芝麻信用分，可以申请大额贷款，转完钱后，第二天联系不上         22.0   
12       在葩趣软件上通过“评论免费送娃”的方式加支付宝好友骗取邮费 被骗的人数已经不少 引起了一定的反应         22.0   


In [9]:
text_len = len(fraud_data)
iter_len = int(text_len/10)

In [4]:
def is_Chinese(word):
    for ch in word:
        if '\u4e00'<=ch<='\u9fff':
            return True
    return False

In [25]:
start = 0
for i in tqdm_notebook(range(text_len // iter_len)):
    fraudi = fraud_data[start:start + iter_len].reset_index(drop=True)
    key_list = ['【','】','套现','http','https']
    for j in range(len(fraudi)):
        text = fraudi['create_memo'].loc[j]
        if not is_Chinese(text):
            fraudi = fraudi.drop(index=j)
            continue
        if any(key in text for key in key_list):
            fraudi = fraudi.drop(index=j)
        if '\n' in text:
            fraudi['create_memo'].loc[j] = re.sub(r'[\'\s]*', '', text)
    fraudi = fraudi.reset_index(drop=True)
    fraudi.to_csv('totaldata/fraudtext_'+str(i)+'.csv',encoding='utf-8')

In [2]:
fraud0 = pd.read_csv('totaldata/fraudtext_2.csv',encoding='utf-8')
fraud0 = fraud0.reset_index(drop=True)
#del fraud0['Unnamed: 0']
fraud0 = fraud0[fraud0['textid']>22566]
fraud0.head()

index  textid     id  is_fraud  finish_first_type  \
4090  13329   22567  70757       1.0               11.0   
4091  13330   22568  70794       1.0               11.0   
4092  13331   22569  70796       1.0               11.0   
4093  13332   22570  70806       1.0               11.0   
4094  13333   22571  70819       1.0               11.0   

                                            create_memo  task_source  \
4090                                对方说代充值话费，扫码付款后就不理我了         22.0   
4091                  是用QQ添加老师，由老师们指路提示操作，在用微信扫码支付宝付款骗取         22.0   
4092  于幸，一开始说是本拥立返的，进了商城加入了购物车点支付宝发送给于幸改价钱，然后我点了那个链接...         22.0   
4093  我是淘宝卖家，找她拿货来卖。钱转了，物品没有给我发货。之前都是用微信转账的，在支付宝给她转了...         22.0   
4094                            对方要求先付定金，剩余其它以快递形式货到付款。         22.0   

       dt_create   scene          dt       ...         \
4090  20190119.0      fp  20190119.0       ...          
4091  20190119.0  direct  20190119.0       ...          
4092  20190119.0  direct  20190119.0       ...          
4093  20190119.0      fp  20190119.0       ...          
4094  20190119.0      fp  20190119.0       ...          

      model_p_complaint_investment_uid_proportion_30d  \
4090                                              NaN   
4091                                              NaN   
4092                                              NaN   
4093                                              NaN   
4094                                              NaN   

      model_p_oppuserid_bereported_usercnt_7d  \
4090                                      NaN   
4091                                      NaN   
4092                                      NaN   
4093                                      NaN   
4094                                      NaN   

      model_p_oppuserid_bereported_usercnt_1d  \
4090                                      NaN   
4091                                      NaN   
4092                                      NaN   
4093                                      NaN   
4094                                      NaN   

      model_p_smid_bereported_amtpercent_1  kw_delete_friend  \
4090                                   NaN               0.0   
4091                                   NaN               0.0   
4092                                   NaN               0.0   
4093                                   NaN               0.0   
4094                                   NaN               0.0   

      kw_fake_document      ocr_dt  \
4090               0.0  20190119.0   
4091               0.0  20190119.0   
4092               0.0  20190119.0   
4093               0.0  20190119.0   
4094               0.0  20190119.0   

                                               ocr_text  swindle_way  \
4090  ["4GH：4Gl 3.5K/SZG蓬洋屏落此人加为好友支付宝收款码好24小时内到账D四@? "]  付款后未收到商品/服务   
4091  ["二维码收款返a二维码收款无需加好友，扫二维码向我付钱?399.00清除金额保存收款码用K...         兼职刷单   
4092  ["\xE4\xB8\xAD\xE5\x9B\xBD\xE7\xA7\xBB\xE5\x8A...         兼职刷单   
4093  ["10:40YUC浏览器<2 阿里巴巴CK工厂店怎么办转不了星期三22:20微信吗Zfb多...  付款后未收到商品/服务   
4094  ["\xE4\xB8\x8A\xE5\x8D\x8811:57ll\xE4\xB8\xAD\...  付款后未收到商品/服务   

      sub_swindle_way  
4090      话费/流量/会员充值等  
4091            其他兼职类  
4092           其他平台刷单  
4093       服饰/手机等实物购买  
4094       服饰/手机等实物购买  

[5 rows x 302 columns]

In [34]:
textid = [i for i in range(len(fraud0))]
fraud0.insert(0,'textid',textid)
fraud0.head()

textid  id  is_fraud  finish_first_type  \
0       0  15       1.0               11.0   
1       1  30       1.0               11.0   
2       2  43       1.0               11.0   
3       3  56       1.0               11.0   
4       4  60       1.0               11.0   

                                         create_memo  task_source   dt_create  \
0                            开始说借后面没就没上线了，现在话都不回一个，唉         22.0  20190115.0   
1  他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0  20190115.0   
2                           骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0  20190115.0   
3                  他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0  20190115.0   
4  他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0  20190115.0   

    scene          dt  score       ...         \
0      fp  20190115.0    0.5       ...          
1      fp  20190115.0    1.0       ...          
2      fp  20190115.0    0.5       ...          
3      FZ  20190115.0    0.0       ...          
4  direct  20190115.0    0.5       ...          

   model_p_complaint_investment_uid_proportion_30d  \
0                                              NaN   
1                                              NaN   
2                                              NaN   
3                                              NaN   
4                                              NaN   

   model_p_oppuserid_bereported_usercnt_7d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_oppuserid_bereported_usercnt_1d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_smid_bereported_amtpercent_1  kw_delete_friend  kw_fake_document  \
0                                   NaN               0.0               0.0   
1                                   NaN               0.0               0.0   
2                                   NaN               0.0               0.0   
3                                   NaN               0.0               0.0   
4                                   NaN               0.0               0.0   

       ocr_dt                                           ocr_text  \
0  20190115.0  ["\xE4\xB8\xAD\xE5\x9B\xBD\xE8\x81\x94\xE9\x80...   
1  20190115.0  ["19:09湿备用金想开的来5分钟之内开湿我支付宝你0.00T始对方佰向话吧好的，等一会、...   
2  20190115.0  ["asahl10.07芯放单品2手情左盐4G999的我时和二个我图一样我格了图一起适你我们...   
3  20190115.0  ["\xE4\xB8\x8B\xE5\x8D\x882\xEF\xBC\x9A26\xE2\...   
4  20190115.0  ["账单详情交易强188.00交易成功191.00订单金融限时-花膜立减3元花呗>付歌方式【...   

    swindle_way  sub_swindle_way  
0  游戏充值/代练等游戏相关            OTHER  
1   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
2          兼职刷单            其他兼职类  
3   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
4          兼职刷单           其他平台刷单  

[5 rows x 301 columns]

textid  id  is_fraud  finish_first_type  \
0       0  15       1.0               11.0   
1       1  30       1.0               11.0   
2       2  43       1.0               11.0   
3       3  56       1.0               11.0   
4       4  60       1.0               11.0   

                                         create_memo  task_source   dt_create  \
0                            开始说借后面没就没上线了，现在话都不回一个，唉         22.0  20190115.0   
1  他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0  20190115.0   
2                           骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0  20190115.0   
3                  他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0  20190115.0   
4  他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0  20190115.0   

    scene          dt  score       ...         \
0      fp  20190115.0    0.5       ...          
1      fp  20190115.0    1.0       ...          
2      fp  20190115.0    0.5       ...          
3      FZ  20190115.0    0.0       ...          
4  direct  20190115.0    0.5       ...          

   model_p_complaint_investment_uid_proportion_30d  \
0                                              NaN   
1                                              NaN   
2                                              NaN   
3                                              NaN   
4                                              NaN   

   model_p_oppuserid_bereported_usercnt_7d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_oppuserid_bereported_usercnt_1d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_smid_bereported_amtpercent_1  kw_delete_friend  kw_fake_document  \
0                                   NaN               0.0               0.0   
1                                   NaN               0.0               0.0   
2                                   NaN               0.0               0.0   
3                                   NaN               0.0               0.0   
4                                   NaN               0.0               0.0   

       ocr_dt                                           ocr_text  \
0  20190115.0  ["\xE4\xB8\xAD\xE5\x9B\xBD\xE8\x81\x94\xE9\x80...   
1  20190115.0  ["19:09湿备用金想开的来5分钟之内开湿我支付宝你0.00T始对方佰向话吧好的，等一会、...   
2  20190115.0  ["asahl10.07芯放单品2手情左盐4G999的我时和二个我图一样我格了图一起适你我们...   
3  20190115.0  ["\xE4\xB8\x8B\xE5\x8D\x882\xEF\xBC\x9A26\xE2\...   
4  20190115.0  ["账单详情交易强188.00交易成功191.00订单金融限时-花膜立减3元花呗>付歌方式【...   

    swindle_way  sub_swindle_way  
0  游戏充值/代练等游戏相关            OTHER  
1   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
2          兼职刷单            其他兼职类  
3   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
4          兼职刷单           其他平台刷单  

[5 rows x 301 columns]

textid  id  is_fraud  finish_first_type  \
0       0  15       1.0               11.0   
1       1  30       1.0               11.0   
2       2  43       1.0               11.0   
3       3  56       1.0               11.0   
4       4  60       1.0               11.0   

                                         create_memo  task_source   dt_create  \
0                            开始说借后面没就没上线了，现在话都不回一个，唉         22.0  20190115.0   
1  他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0  20190115.0   
2                           骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0  20190115.0   
3                  他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0  20190115.0   
4  他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0  20190115.0   

    scene          dt  score       ...         \
0      fp  20190115.0    0.5       ...          
1      fp  20190115.0    1.0       ...          
2      fp  20190115.0    0.5       ...          
3      FZ  20190115.0    0.0       ...          
4  direct  20190115.0    0.5       ...          

   model_p_complaint_investment_uid_proportion_30d  \
0                                              NaN   
1                                              NaN   
2                                              NaN   
3                                              NaN   
4                                              NaN   

   model_p_oppuserid_bereported_usercnt_7d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_oppuserid_bereported_usercnt_1d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_smid_bereported_amtpercent_1  kw_delete_friend  kw_fake_document  \
0                                   NaN               0.0               0.0   
1                                   NaN               0.0               0.0   
2                                   NaN               0.0               0.0   
3                                   NaN               0.0               0.0   
4                                   NaN               0.0               0.0   

       ocr_dt                                           ocr_text  \
0  20190115.0  ["\xE4\xB8\xAD\xE5\x9B\xBD\xE8\x81\x94\xE9\x80...   
1  20190115.0  ["19:09湿备用金想开的来5分钟之内开湿我支付宝你0.00T始对方佰向话吧好的，等一会、...   
2  20190115.0  ["asahl10.07芯放单品2手情左盐4G999的我时和二个我图一样我格了图一起适你我们...   
3  20190115.0  ["\xE4\xB8\x8B\xE5\x8D\x882\xEF\xBC\x9A26\xE2\...   
4  20190115.0  ["账单详情交易强188.00交易成功191.00订单金融限时-花膜立减3元花呗>付歌方式【...   

    swindle_way  sub_swindle_way  
0  游戏充值/代练等游戏相关            OTHER  
1   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
2          兼职刷单            其他兼职类  
3   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
4          兼职刷单           其他平台刷单  

[5 rows x 301 columns]

textid  id  is_fraud  finish_first_type  \
0       0  15       1.0               11.0   
1       1  30       1.0               11.0   
2       2  43       1.0               11.0   
3       3  56       1.0               11.0   
4       4  60       1.0               11.0   

                                         create_memo  task_source   dt_create  \
0                            开始说借后面没就没上线了，现在话都不回一个，唉         22.0  20190115.0   
1  他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0  20190115.0   
2                           骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0  20190115.0   
3                  他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0  20190115.0   
4  他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0  20190115.0   

    scene          dt  score       ...         \
0      fp  20190115.0    0.5       ...          
1      fp  20190115.0    1.0       ...          
2      fp  20190115.0    0.5       ...          
3      FZ  20190115.0    0.0       ...          
4  direct  20190115.0    0.5       ...          

   model_p_complaint_investment_uid_proportion_30d  \
0                                              NaN   
1                                              NaN   
2                                              NaN   
3                                              NaN   
4                                              NaN   

   model_p_oppuserid_bereported_usercnt_7d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_oppuserid_bereported_usercnt_1d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_smid_bereported_amtpercent_1  kw_delete_friend  kw_fake_document  \
0                                   NaN               0.0               0.0   
1                                   NaN               0.0               0.0   
2                                   NaN               0.0               0.0   
3                                   NaN               0.0               0.0   
4                                   NaN               0.0               0.0   

       ocr_dt                                           ocr_text  \
0  20190115.0  ["\xE4\xB8\xAD\xE5\x9B\xBD\xE8\x81\x94\xE9\x80...   
1  20190115.0  ["19:09湿备用金想开的来5分钟之内开湿我支付宝你0.00T始对方佰向话吧好的，等一会、...   
2  20190115.0  ["asahl10.07芯放单品2手情左盐4G999的我时和二个我图一样我格了图一起适你我们...   
3  20190115.0  ["\xE4\xB8\x8B\xE5\x8D\x882\xEF\xBC\x9A26\xE2\...   
4  20190115.0  ["账单详情交易强188.00交易成功191.00订单金融限时-花膜立减3元花呗>付歌方式【...   

    swindle_way  sub_swindle_way  
0  游戏充值/代练等游戏相关            OTHER  
1   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
2          兼职刷单            其他兼职类  
3   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
4          兼职刷单           其他平台刷单  

[5 rows x 301 columns]

textid  id  is_fraud  finish_first_type  \
0       0  15       1.0               11.0   
1       1  30       1.0               11.0   
2       2  43       1.0               11.0   
3       3  56       1.0               11.0   
4       4  60       1.0               11.0   

                                         create_memo  task_source   dt_create  \
0                            开始说借后面没就没上线了，现在话都不回一个，唉         22.0  20190115.0   
1  他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0  20190115.0   
2                           骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0  20190115.0   
3                  他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0  20190115.0   
4  他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0  20190115.0   

    scene          dt  score       ...         \
0      fp  20190115.0    0.5       ...          
1      fp  20190115.0    1.0       ...          
2      fp  20190115.0    0.5       ...          
3      FZ  20190115.0    0.0       ...          
4  direct  20190115.0    0.5       ...          

   model_p_complaint_investment_uid_proportion_30d  \
0                                              NaN   
1                                              NaN   
2                                              NaN   
3                                              NaN   
4                                              NaN   

   model_p_oppuserid_bereported_usercnt_7d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_oppuserid_bereported_usercnt_1d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_smid_bereported_amtpercent_1  kw_delete_friend  kw_fake_document  \
0                                   NaN               0.0               0.0   
1                                   NaN               0.0               0.0   
2                                   NaN               0.0               0.0   
3                                   NaN               0.0               0.0   
4                                   NaN               0.0               0.0   

       ocr_dt                                           ocr_text  \
0  20190115.0  ["\xE4\xB8\xAD\xE5\x9B\xBD\xE8\x81\x94\xE9\x80...   
1  20190115.0  ["19:09湿备用金想开的来5分钟之内开湿我支付宝你0.00T始对方佰向话吧好的，等一会、...   
2  20190115.0  ["asahl10.07芯放单品2手情左盐4G999的我时和二个我图一样我格了图一起适你我们...   
3  20190115.0  ["\xE4\xB8\x8B\xE5\x8D\x882\xEF\xBC\x9A26\xE2\...   
4  20190115.0  ["账单详情交易强188.00交易成功191.00订单金融限时-花膜立减3元花呗>付歌方式【...   

    swindle_way  sub_swindle_way  
0  游戏充值/代练等游戏相关            OTHER  
1   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
2          兼职刷单            其他兼职类  
3   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
4          兼职刷单           其他平台刷单  

[5 rows x 301 columns]

textid  id  is_fraud  finish_first_type  \
0       0  15       1.0               11.0   
1       1  30       1.0               11.0   
2       2  43       1.0               11.0   
3       3  56       1.0               11.0   
4       4  60       1.0               11.0   

                                         create_memo  task_source   dt_create  \
0                            开始说借后面没就没上线了，现在话都不回一个，唉         22.0  20190115.0   
1  他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0  20190115.0   
2                           骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0  20190115.0   
3                  他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0  20190115.0   
4  他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0  20190115.0   

    scene          dt  score       ...         \
0      fp  20190115.0    0.5       ...          
1      fp  20190115.0    1.0       ...          
2      fp  20190115.0    0.5       ...          
3      FZ  20190115.0    0.0       ...          
4  direct  20190115.0    0.5       ...          

   model_p_complaint_investment_uid_proportion_30d  \
0                                              NaN   
1                                              NaN   
2                                              NaN   
3                                              NaN   
4                                              NaN   

   model_p_oppuserid_bereported_usercnt_7d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_oppuserid_bereported_usercnt_1d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_smid_bereported_amtpercent_1  kw_delete_friend  kw_fake_document  \
0                                   NaN               0.0               0.0   
1                                   NaN               0.0               0.0   
2                                   NaN               0.0               0.0   
3                                   NaN               0.0               0.0   
4                                   NaN               0.0               0.0   

       ocr_dt                                           ocr_text  \
0  20190115.0  ["\xE4\xB8\xAD\xE5\x9B\xBD\xE8\x81\x94\xE9\x80...   
1  20190115.0  ["19:09湿备用金想开的来5分钟之内开湿我支付宝你0.00T始对方佰向话吧好的，等一会、...   
2  20190115.0  ["asahl10.07芯放单品2手情左盐4G999的我时和二个我图一样我格了图一起适你我们...   
3  20190115.0  ["\xE4\xB8\x8B\xE5\x8D\x882\xEF\xBC\x9A26\xE2\...   
4  20190115.0  ["账单详情交易强188.00交易成功191.00订单金融限时-花膜立减3元花呗>付歌方式【...   

    swindle_way  sub_swindle_way  
0  游戏充值/代练等游戏相关            OTHER  
1   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
2          兼职刷单            其他兼职类  
3   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
4          兼职刷单           其他平台刷单  

[5 rows x 301 columns]

textid  id  is_fraud  finish_first_type  \
0       0  15       1.0               11.0   
1       1  30       1.0               11.0   
2       2  43       1.0               11.0   
3       3  56       1.0               11.0   
4       4  60       1.0               11.0   

                                         create_memo  task_source   dt_create  \
0                            开始说借后面没就没上线了，现在话都不回一个，唉         22.0  20190115.0   
1  他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0  20190115.0   
2                           骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0  20190115.0   
3                  他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0  20190115.0   
4  他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0  20190115.0   

    scene          dt  score       ...         \
0      fp  20190115.0    0.5       ...          
1      fp  20190115.0    1.0       ...          
2      fp  20190115.0    0.5       ...          
3      FZ  20190115.0    0.0       ...          
4  direct  20190115.0    0.5       ...          

   model_p_complaint_investment_uid_proportion_30d  \
0                                              NaN   
1                                              NaN   
2                                              NaN   
3                                              NaN   
4                                              NaN   

   model_p_oppuserid_bereported_usercnt_7d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_oppuserid_bereported_usercnt_1d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_smid_bereported_amtpercent_1  kw_delete_friend  kw_fake_document  \
0                                   NaN               0.0               0.0   
1                                   NaN               0.0               0.0   
2                                   NaN               0.0               0.0   
3                                   NaN               0.0               0.0   
4                                   NaN               0.0               0.0   

       ocr_dt                                           ocr_text  \
0  20190115.0  ["\xE4\xB8\xAD\xE5\x9B\xBD\xE8\x81\x94\xE9\x80...   
1  20190115.0  ["19:09湿备用金想开的来5分钟之内开湿我支付宝你0.00T始对方佰向话吧好的，等一会、...   
2  20190115.0  ["asahl10.07芯放单品2手情左盐4G999的我时和二个我图一样我格了图一起适你我们...   
3  20190115.0  ["\xE4\xB8\x8B\xE5\x8D\x882\xEF\xBC\x9A26\xE2\...   
4  20190115.0  ["账单详情交易强188.00交易成功191.00订单金融限时-花膜立减3元花呗>付歌方式【...   

    swindle_way  sub_swindle_way  
0  游戏充值/代练等游戏相关            OTHER  
1   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
2          兼职刷单            其他兼职类  
3   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
4          兼职刷单           其他平台刷单  

[5 rows x 301 columns]

In [35]:
fraud0['textid'] = fraud0['textid']+78150
fraud0.head()

textid  id  is_fraud  finish_first_type  \
0   78150  15       1.0               11.0   
1   78151  30       1.0               11.0   
2   78152  43       1.0               11.0   
3   78153  56       1.0               11.0   
4   78154  60       1.0               11.0   

                                         create_memo  task_source   dt_create  \
0                            开始说借后面没就没上线了，现在话都不回一个，唉         22.0  20190115.0   
1  他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0  20190115.0   
2                           骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0  20190115.0   
3                  他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0  20190115.0   
4  他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0  20190115.0   

    scene          dt  score       ...         \
0      fp  20190115.0    0.5       ...          
1      fp  20190115.0    1.0       ...          
2      fp  20190115.0    0.5       ...          
3      FZ  20190115.0    0.0       ...          
4  direct  20190115.0    0.5       ...          

   model_p_complaint_investment_uid_proportion_30d  \
0                                              NaN   
1                                              NaN   
2                                              NaN   
3                                              NaN   
4                                              NaN   

   model_p_oppuserid_bereported_usercnt_7d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_oppuserid_bereported_usercnt_1d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_smid_bereported_amtpercent_1  kw_delete_friend  kw_fake_document  \
0                                   NaN               0.0               0.0   
1                                   NaN               0.0               0.0   
2                                   NaN               0.0               0.0   
3                                   NaN               0.0               0.0   
4                                   NaN               0.0               0.0   

       ocr_dt                                           ocr_text  \
0  20190115.0  ["\xE4\xB8\xAD\xE5\x9B\xBD\xE8\x81\x94\xE9\x80...   
1  20190115.0  ["19:09湿备用金想开的来5分钟之内开湿我支付宝你0.00T始对方佰向话吧好的，等一会、...   
2  20190115.0  ["asahl10.07芯放单品2手情左盐4G999的我时和二个我图一样我格了图一起适你我们...   
3  20190115.0  ["\xE4\xB8\x8B\xE5\x8D\x882\xEF\xBC\x9A26\xE2\...   
4  20190115.0  ["账单详情交易强188.00交易成功191.00订单金融限时-花膜立减3元花呗>付歌方式【...   

    swindle_way  sub_swindle_way  
0  游戏充值/代练等游戏相关            OTHER  
1   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
2          兼职刷单            其他兼职类  
3   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
4          兼职刷单           其他平台刷单  

[5 rows x 301 columns]

textid  id  is_fraud  finish_first_type  \
0   78150  15       1.0               11.0   
1   78151  30       1.0               11.0   
2   78152  43       1.0               11.0   
3   78153  56       1.0               11.0   
4   78154  60       1.0               11.0   

                                         create_memo  task_source   dt_create  \
0                            开始说借后面没就没上线了，现在话都不回一个，唉         22.0  20190115.0   
1  他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0  20190115.0   
2                           骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0  20190115.0   
3                  他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0  20190115.0   
4  他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0  20190115.0   

    scene          dt  score       ...         \
0      fp  20190115.0    0.5       ...          
1      fp  20190115.0    1.0       ...          
2      fp  20190115.0    0.5       ...          
3      FZ  20190115.0    0.0       ...          
4  direct  20190115.0    0.5       ...          

   model_p_complaint_investment_uid_proportion_30d  \
0                                              NaN   
1                                              NaN   
2                                              NaN   
3                                              NaN   
4                                              NaN   

   model_p_oppuserid_bereported_usercnt_7d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_oppuserid_bereported_usercnt_1d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_smid_bereported_amtpercent_1  kw_delete_friend  kw_fake_document  \
0                                   NaN               0.0               0.0   
1                                   NaN               0.0               0.0   
2                                   NaN               0.0               0.0   
3                                   NaN               0.0               0.0   
4                                   NaN               0.0               0.0   

       ocr_dt                                           ocr_text  \
0  20190115.0  ["\xE4\xB8\xAD\xE5\x9B\xBD\xE8\x81\x94\xE9\x80...   
1  20190115.0  ["19:09湿备用金想开的来5分钟之内开湿我支付宝你0.00T始对方佰向话吧好的，等一会、...   
2  20190115.0  ["asahl10.07芯放单品2手情左盐4G999的我时和二个我图一样我格了图一起适你我们...   
3  20190115.0  ["\xE4\xB8\x8B\xE5\x8D\x882\xEF\xBC\x9A26\xE2\...   
4  20190115.0  ["账单详情交易强188.00交易成功191.00订单金融限时-花膜立减3元花呗>付歌方式【...   

    swindle_way  sub_swindle_way  
0  游戏充值/代练等游戏相关            OTHER  
1   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
2          兼职刷单            其他兼职类  
3   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
4          兼职刷单           其他平台刷单  

[5 rows x 301 columns]

textid  id  is_fraud  finish_first_type  \
0   78150  15       1.0               11.0   
1   78151  30       1.0               11.0   
2   78152  43       1.0               11.0   
3   78153  56       1.0               11.0   
4   78154  60       1.0               11.0   

                                         create_memo  task_source   dt_create  \
0                            开始说借后面没就没上线了，现在话都不回一个，唉         22.0  20190115.0   
1  他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0  20190115.0   
2                           骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0  20190115.0   
3                  他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0  20190115.0   
4  他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0  20190115.0   

    scene          dt  score       ...         \
0      fp  20190115.0    0.5       ...          
1      fp  20190115.0    1.0       ...          
2      fp  20190115.0    0.5       ...          
3      FZ  20190115.0    0.0       ...          
4  direct  20190115.0    0.5       ...          

   model_p_complaint_investment_uid_proportion_30d  \
0                                              NaN   
1                                              NaN   
2                                              NaN   
3                                              NaN   
4                                              NaN   

   model_p_oppuserid_bereported_usercnt_7d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_oppuserid_bereported_usercnt_1d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_smid_bereported_amtpercent_1  kw_delete_friend  kw_fake_document  \
0                                   NaN               0.0               0.0   
1                                   NaN               0.0               0.0   
2                                   NaN               0.0               0.0   
3                                   NaN               0.0               0.0   
4                                   NaN               0.0               0.0   

       ocr_dt                                           ocr_text  \
0  20190115.0  ["\xE4\xB8\xAD\xE5\x9B\xBD\xE8\x81\x94\xE9\x80...   
1  20190115.0  ["19:09湿备用金想开的来5分钟之内开湿我支付宝你0.00T始对方佰向话吧好的，等一会、...   
2  20190115.0  ["asahl10.07芯放单品2手情左盐4G999的我时和二个我图一样我格了图一起适你我们...   
3  20190115.0  ["\xE4\xB8\x8B\xE5\x8D\x882\xEF\xBC\x9A26\xE2\...   
4  20190115.0  ["账单详情交易强188.00交易成功191.00订单金融限时-花膜立减3元花呗>付歌方式【...   

    swindle_way  sub_swindle_way  
0  游戏充值/代练等游戏相关            OTHER  
1   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
2          兼职刷单            其他兼职类  
3   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
4          兼职刷单           其他平台刷单  

[5 rows x 301 columns]

textid  id  is_fraud  finish_first_type  \
0   78150  15       1.0               11.0   
1   78151  30       1.0               11.0   
2   78152  43       1.0               11.0   
3   78153  56       1.0               11.0   
4   78154  60       1.0               11.0   

                                         create_memo  task_source   dt_create  \
0                            开始说借后面没就没上线了，现在话都不回一个，唉         22.0  20190115.0   
1  他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0  20190115.0   
2                           骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0  20190115.0   
3                  他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0  20190115.0   
4  他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0  20190115.0   

    scene          dt  score       ...         \
0      fp  20190115.0    0.5       ...          
1      fp  20190115.0    1.0       ...          
2      fp  20190115.0    0.5       ...          
3      FZ  20190115.0    0.0       ...          
4  direct  20190115.0    0.5       ...          

   model_p_complaint_investment_uid_proportion_30d  \
0                                              NaN   
1                                              NaN   
2                                              NaN   
3                                              NaN   
4                                              NaN   

   model_p_oppuserid_bereported_usercnt_7d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_oppuserid_bereported_usercnt_1d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_smid_bereported_amtpercent_1  kw_delete_friend  kw_fake_document  \
0                                   NaN               0.0               0.0   
1                                   NaN               0.0               0.0   
2                                   NaN               0.0               0.0   
3                                   NaN               0.0               0.0   
4                                   NaN               0.0               0.0   

       ocr_dt                                           ocr_text  \
0  20190115.0  ["\xE4\xB8\xAD\xE5\x9B\xBD\xE8\x81\x94\xE9\x80...   
1  20190115.0  ["19:09湿备用金想开的来5分钟之内开湿我支付宝你0.00T始对方佰向话吧好的，等一会、...   
2  20190115.0  ["asahl10.07芯放单品2手情左盐4G999的我时和二个我图一样我格了图一起适你我们...   
3  20190115.0  ["\xE4\xB8\x8B\xE5\x8D\x882\xEF\xBC\x9A26\xE2\...   
4  20190115.0  ["账单详情交易强188.00交易成功191.00订单金融限时-花膜立减3元花呗>付歌方式【...   

    swindle_way  sub_swindle_way  
0  游戏充值/代练等游戏相关            OTHER  
1   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
2          兼职刷单            其他兼职类  
3   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
4          兼职刷单           其他平台刷单  

[5 rows x 301 columns]

textid  id  is_fraud  finish_first_type  \
0   78150  15       1.0               11.0   
1   78151  30       1.0               11.0   
2   78152  43       1.0               11.0   
3   78153  56       1.0               11.0   
4   78154  60       1.0               11.0   

                                         create_memo  task_source   dt_create  \
0                            开始说借后面没就没上线了，现在话都不回一个，唉         22.0  20190115.0   
1  他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0  20190115.0   
2                           骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0  20190115.0   
3                  他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0  20190115.0   
4  他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0  20190115.0   

    scene          dt  score       ...         \
0      fp  20190115.0    0.5       ...          
1      fp  20190115.0    1.0       ...          
2      fp  20190115.0    0.5       ...          
3      FZ  20190115.0    0.0       ...          
4  direct  20190115.0    0.5       ...          

   model_p_complaint_investment_uid_proportion_30d  \
0                                              NaN   
1                                              NaN   
2                                              NaN   
3                                              NaN   
4                                              NaN   

   model_p_oppuserid_bereported_usercnt_7d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_oppuserid_bereported_usercnt_1d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_smid_bereported_amtpercent_1  kw_delete_friend  kw_fake_document  \
0                                   NaN               0.0               0.0   
1                                   NaN               0.0               0.0   
2                                   NaN               0.0               0.0   
3                                   NaN               0.0               0.0   
4                                   NaN               0.0               0.0   

       ocr_dt                                           ocr_text  \
0  20190115.0  ["\xE4\xB8\xAD\xE5\x9B\xBD\xE8\x81\x94\xE9\x80...   
1  20190115.0  ["19:09湿备用金想开的来5分钟之内开湿我支付宝你0.00T始对方佰向话吧好的，等一会、...   
2  20190115.0  ["asahl10.07芯放单品2手情左盐4G999的我时和二个我图一样我格了图一起适你我们...   
3  20190115.0  ["\xE4\xB8\x8B\xE5\x8D\x882\xEF\xBC\x9A26\xE2\...   
4  20190115.0  ["账单详情交易强188.00交易成功191.00订单金融限时-花膜立减3元花呗>付歌方式【...   

    swindle_way  sub_swindle_way  
0  游戏充值/代练等游戏相关            OTHER  
1   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
2          兼职刷单            其他兼职类  
3   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
4          兼职刷单           其他平台刷单  

[5 rows x 301 columns]

textid  id  is_fraud  finish_first_type  \
0   78150  15       1.0               11.0   
1   78151  30       1.0               11.0   
2   78152  43       1.0               11.0   
3   78153  56       1.0               11.0   
4   78154  60       1.0               11.0   

                                         create_memo  task_source   dt_create  \
0                            开始说借后面没就没上线了，现在话都不回一个，唉         22.0  20190115.0   
1  他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0  20190115.0   
2                           骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0  20190115.0   
3                  他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0  20190115.0   
4  他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0  20190115.0   

    scene          dt  score       ...         \
0      fp  20190115.0    0.5       ...          
1      fp  20190115.0    1.0       ...          
2      fp  20190115.0    0.5       ...          
3      FZ  20190115.0    0.0       ...          
4  direct  20190115.0    0.5       ...          

   model_p_complaint_investment_uid_proportion_30d  \
0                                              NaN   
1                                              NaN   
2                                              NaN   
3                                              NaN   
4                                              NaN   

   model_p_oppuserid_bereported_usercnt_7d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_oppuserid_bereported_usercnt_1d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_smid_bereported_amtpercent_1  kw_delete_friend  kw_fake_document  \
0                                   NaN               0.0               0.0   
1                                   NaN               0.0               0.0   
2                                   NaN               0.0               0.0   
3                                   NaN               0.0               0.0   
4                                   NaN               0.0               0.0   

       ocr_dt                                           ocr_text  \
0  20190115.0  ["\xE4\xB8\xAD\xE5\x9B\xBD\xE8\x81\x94\xE9\x80...   
1  20190115.0  ["19:09湿备用金想开的来5分钟之内开湿我支付宝你0.00T始对方佰向话吧好的，等一会、...   
2  20190115.0  ["asahl10.07芯放单品2手情左盐4G999的我时和二个我图一样我格了图一起适你我们...   
3  20190115.0  ["\xE4\xB8\x8B\xE5\x8D\x882\xEF\xBC\x9A26\xE2\...   
4  20190115.0  ["账单详情交易强188.00交易成功191.00订单金融限时-花膜立减3元花呗>付歌方式【...   

    swindle_way  sub_swindle_way  
0  游戏充值/代练等游戏相关            OTHER  
1   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
2          兼职刷单            其他兼职类  
3   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
4          兼职刷单           其他平台刷单  

[5 rows x 301 columns]

textid  id  is_fraud  finish_first_type  \
0   78150  15       1.0               11.0   
1   78151  30       1.0               11.0   
2   78152  43       1.0               11.0   
3   78153  56       1.0               11.0   
4   78154  60       1.0               11.0   

                                         create_memo  task_source   dt_create  \
0                            开始说借后面没就没上线了，现在话都不回一个，唉         22.0  20190115.0   
1  他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0  20190115.0   
2                           骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0  20190115.0   
3                  他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0  20190115.0   
4  他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0  20190115.0   

    scene          dt  score       ...         \
0      fp  20190115.0    0.5       ...          
1      fp  20190115.0    1.0       ...          
2      fp  20190115.0    0.5       ...          
3      FZ  20190115.0    0.0       ...          
4  direct  20190115.0    0.5       ...          

   model_p_complaint_investment_uid_proportion_30d  \
0                                              NaN   
1                                              NaN   
2                                              NaN   
3                                              NaN   
4                                              NaN   

   model_p_oppuserid_bereported_usercnt_7d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_oppuserid_bereported_usercnt_1d  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   model_p_smid_bereported_amtpercent_1  kw_delete_friend  kw_fake_document  \
0                                   NaN               0.0               0.0   
1                                   NaN               0.0               0.0   
2                                   NaN               0.0               0.0   
3                                   NaN               0.0               0.0   
4                                   NaN               0.0               0.0   

       ocr_dt                                           ocr_text  \
0  20190115.0  ["\xE4\xB8\xAD\xE5\x9B\xBD\xE8\x81\x94\xE9\x80...   
1  20190115.0  ["19:09湿备用金想开的来5分钟之内开湿我支付宝你0.00T始对方佰向话吧好的，等一会、...   
2  20190115.0  ["asahl10.07芯放单品2手情左盐4G999的我时和二个我图一样我格了图一起适你我们...   
3  20190115.0  ["\xE4\xB8\x8B\xE5\x8D\x882\xEF\xBC\x9A26\xE2\...   
4  20190115.0  ["账单详情交易强188.00交易成功191.00订单金融限时-花膜立减3元花呗>付歌方式【...   

    swindle_way  sub_swindle_way  
0  游戏充值/代练等游戏相关            OTHER  
1   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
2          兼职刷单            其他兼职类  
3   贷款/信用卡开通、提额      贷款/办卡/提额类其他  
4          兼职刷单           其他平台刷单  

[5 rows x 301 columns]

In [3]:
fraud0[['index','textid','create_memo']].to_csv('totaldata/fraud_2.csv',encoding='utf-8',index=False)

In [54]:
before = pd.read_csv('totaldata/fraudtext_3.csv',encoding='utf-8')
for i in range(4,10):
    fraud = pd.read_csv('totaldata/fraudtext_'+str(i)+'.csv',encoding='utf-8')
    before = pd.concat([before,fraud]).sort_values(by='textid').reset_index(drop=True)

print(len(before))

64673646736467364673646736467364673








In [55]:
del before['index']
before

textid      id  is_fraud  finish_first_type  \
0       22716      15       1.0               11.0   
1       22717      30       1.0               11.0   
2       22718      43       1.0               11.0   
3       22719      56       1.0               11.0   
4       22720      60       1.0               11.0   
5       22721      64       1.0               11.0   
6       22722     119       1.0               11.0   
7       22723     121       1.0               11.0   
8       22724     158       1.0               11.0   
9       22725     179       1.0               11.0   
10      22726     222       1.0               11.0   
11      22727     303       1.0               11.0   
12      22728     325       1.0               11.0   
13      22729     346       1.0               11.0   
14      22730     406       1.0               11.0   
15      22731     422       1.0               11.0   
16      22732     435       1.0               11.0   
17      22733     470       1.0               11.0   
18      22734     479       1.0               11.0   
19      22735     490       1.0               11.0   
20      22736     520       1.0               11.0   
21      22737     542       1.0               11.0   
22      22738     578       1.0               11.0   
23      22739     668       1.0               11.0   
24      22740     765       1.0               11.0   
25      22741     826       1.0               11.0   
26      22742     831       1.0               11.0   
27      22743     832       1.0               11.0   
28      22744     945       1.0               11.0   
29      22745     984       1.0               11.0   
...       ...     ...       ...                ...   
64643   87359  156353       1.0               11.0   
64644   87360  156365       1.0               11.0   
64645   87361  156366       1.0               11.0   
64646   87362  156373       1.0               11.0   
64647   87363  156375       1.0               11.0   
64648   87364  156376       1.0               11.0   
64649   87365  156391       1.0               11.0   
64650   87366  156394       1.0               11.0   
64651   87367  156403       1.0               11.0   
64652   87368  156418       1.0               11.0   
64653   87369  156433       1.0               11.0   
64654   87370  156442       1.0               11.0   
64655   87371  156446       1.0               11.0   
64656   87372  156449       1.0               11.0   
64657   87373  156480       1.0               11.0   
64658   87374  156482       1.0               11.0   
64659   87375  156505       1.0               11.0   
64660   87376  156526       1.0               11.0   
64661   87377  156534       1.0               11.0   
64662   87378  156545       1.0               11.0   
64663   87379  156548       1.0               11.0   
64664   87380  156551       1.0               11.0   
64665   87381  156575       1.0               11.0   
64666   87382  156594       1.0               11.0   
64667   87383  156597       1.0               11.0   
64668   87384  156610       1.0               11.0   
64669   87385  156611       1.0               11.0   
64670   87386  156612       1.0               11.0   
64671   87387  156623       1.0               11.0   
64672   87388  156624       1.0               11.0   

                                             create_memo  task_source  \
0                                开始说借后面没就没上线了，现在话都不回一个，唉         22.0   
1      他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0   
2                               骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0   
3                      他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0   
4      他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0   
5                        这是交的主持会费，交完后我就后悔了叫她返会费，就直接拉黑我了。         22.0   
6                          他以玩游戏充值送分为由，骗了我五千元，结果人都联系不上了。         22.0   
7      盗用我朋友的QQ号像我骗取4000元钱，一次是通过微信扫描二维码转账给对方1200，一次是通...         22.0   
8

textid      id  is_fraud  finish_first_type  \
0       22716      15       1.0               11.0   
1       22717      30       1.0               11.0   
2       22718      43       1.0               11.0   
3       22719      56       1.0               11.0   
4       22720      60       1.0               11.0   
5       22721      64       1.0               11.0   
6       22722     119       1.0               11.0   
7       22723     121       1.0               11.0   
8       22724     158       1.0               11.0   
9       22725     179       1.0               11.0   
10      22726     222       1.0               11.0   
11      22727     303       1.0               11.0   
12      22728     325       1.0               11.0   
13      22729     346       1.0               11.0   
14      22730     406       1.0               11.0   
15      22731     422       1.0               11.0   
16      22732     435       1.0               11.0   
17      22733     470       1.0               11.0   
18      22734     479       1.0               11.0   
19      22735     490       1.0               11.0   
20      22736     520       1.0               11.0   
21      22737     542       1.0               11.0   
22      22738     578       1.0               11.0   
23      22739     668       1.0               11.0   
24      22740     765       1.0               11.0   
25      22741     826       1.0               11.0   
26      22742     831       1.0               11.0   
27      22743     832       1.0               11.0   
28      22744     945       1.0               11.0   
29      22745     984       1.0               11.0   
...       ...     ...       ...                ...   
64643   87359  156353       1.0               11.0   
64644   87360  156365       1.0               11.0   
64645   87361  156366       1.0               11.0   
64646   87362  156373       1.0               11.0   
64647   87363  156375       1.0               11.0   
64648   87364  156376       1.0               11.0   
64649   87365  156391       1.0               11.0   
64650   87366  156394       1.0               11.0   
64651   87367  156403       1.0               11.0   
64652   87368  156418       1.0               11.0   
64653   87369  156433       1.0               11.0   
64654   87370  156442       1.0               11.0   
64655   87371  156446       1.0               11.0   
64656   87372  156449       1.0               11.0   
64657   87373  156480       1.0               11.0   
64658   87374  156482       1.0               11.0   
64659   87375  156505       1.0               11.0   
64660   87376  156526       1.0               11.0   
64661   87377  156534       1.0               11.0   
64662   87378  156545       1.0               11.0   
64663   87379  156548       1.0               11.0   
64664   87380  156551       1.0               11.0   
64665   87381  156575       1.0               11.0   
64666   87382  156594       1.0               11.0   
64667   87383  156597       1.0               11.0   
64668   87384  156610       1.0               11.0   
64669   87385  156611       1.0               11.0   
64670   87386  156612       1.0               11.0   
64671   87387  156623       1.0               11.0   
64672   87388  156624       1.0               11.0   

                                             create_memo  task_source  \
0                                开始说借后面没就没上线了，现在话都不回一个，唉         22.0   
1      他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0   
2                               骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0   
3                      他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0   
4      他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0   
5                        这是交的主持会费，交完后我就后悔了叫她返会费，就直接拉黑我了。         22.0   
6                          他以玩游戏充值送分为由，骗了我五千元，结果人都联系不上了。         22.0   
7      盗用我朋友的QQ号像我骗取4000元钱，一次是通过微信扫描二维码转账给对方1200，一次是通...         22.0   
8

textid      id  is_fraud  finish_first_type  \
0       22716      15       1.0               11.0   
1       22717      30       1.0               11.0   
2       22718      43       1.0               11.0   
3       22719      56       1.0               11.0   
4       22720      60       1.0               11.0   
5       22721      64       1.0               11.0   
6       22722     119       1.0               11.0   
7       22723     121       1.0               11.0   
8       22724     158       1.0               11.0   
9       22725     179       1.0               11.0   
10      22726     222       1.0               11.0   
11      22727     303       1.0               11.0   
12      22728     325       1.0               11.0   
13      22729     346       1.0               11.0   
14      22730     406       1.0               11.0   
15      22731     422       1.0               11.0   
16      22732     435       1.0               11.0   
17      22733     470       1.0               11.0   
18      22734     479       1.0               11.0   
19      22735     490       1.0               11.0   
20      22736     520       1.0               11.0   
21      22737     542       1.0               11.0   
22      22738     578       1.0               11.0   
23      22739     668       1.0               11.0   
24      22740     765       1.0               11.0   
25      22741     826       1.0               11.0   
26      22742     831       1.0               11.0   
27      22743     832       1.0               11.0   
28      22744     945       1.0               11.0   
29      22745     984       1.0               11.0   
...       ...     ...       ...                ...   
64643   87359  156353       1.0               11.0   
64644   87360  156365       1.0               11.0   
64645   87361  156366       1.0               11.0   
64646   87362  156373       1.0               11.0   
64647   87363  156375       1.0               11.0   
64648   87364  156376       1.0               11.0   
64649   87365  156391       1.0               11.0   
64650   87366  156394       1.0               11.0   
64651   87367  156403       1.0               11.0   
64652   87368  156418       1.0               11.0   
64653   87369  156433       1.0               11.0   
64654   87370  156442       1.0               11.0   
64655   87371  156446       1.0               11.0   
64656   87372  156449       1.0               11.0   
64657   87373  156480       1.0               11.0   
64658   87374  156482       1.0               11.0   
64659   87375  156505       1.0               11.0   
64660   87376  156526       1.0               11.0   
64661   87377  156534       1.0               11.0   
64662   87378  156545       1.0               11.0   
64663   87379  156548       1.0               11.0   
64664   87380  156551       1.0               11.0   
64665   87381  156575       1.0               11.0   
64666   87382  156594       1.0               11.0   
64667   87383  156597       1.0               11.0   
64668   87384  156610       1.0               11.0   
64669   87385  156611       1.0               11.0   
64670   87386  156612       1.0               11.0   
64671   87387  156623       1.0               11.0   
64672   87388  156624       1.0               11.0   

                                             create_memo  task_source  \
0                                开始说借后面没就没上线了，现在话都不回一个，唉         22.0   
1      他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0   
2                               骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0   
3                      他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0   
4      他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0   
5                        这是交的主持会费，交完后我就后悔了叫她返会费，就直接拉黑我了。         22.0   
6                          他以玩游戏充值送分为由，骗了我五千元，结果人都联系不上了。         22.0   
7      盗用我朋友的QQ号像我骗取4000元钱，一次是通过微信扫描二维码转账给对方1200，一次是通...         22.0   
8

textid      id  is_fraud  finish_first_type  \
0       22716      15       1.0               11.0   
1       22717      30       1.0               11.0   
2       22718      43       1.0               11.0   
3       22719      56       1.0               11.0   
4       22720      60       1.0               11.0   
5       22721      64       1.0               11.0   
6       22722     119       1.0               11.0   
7       22723     121       1.0               11.0   
8       22724     158       1.0               11.0   
9       22725     179       1.0               11.0   
10      22726     222       1.0               11.0   
11      22727     303       1.0               11.0   
12      22728     325       1.0               11.0   
13      22729     346       1.0               11.0   
14      22730     406       1.0               11.0   
15      22731     422       1.0               11.0   
16      22732     435       1.0               11.0   
17      22733     470       1.0               11.0   
18      22734     479       1.0               11.0   
19      22735     490       1.0               11.0   
20      22736     520       1.0               11.0   
21      22737     542       1.0               11.0   
22      22738     578       1.0               11.0   
23      22739     668       1.0               11.0   
24      22740     765       1.0               11.0   
25      22741     826       1.0               11.0   
26      22742     831       1.0               11.0   
27      22743     832       1.0               11.0   
28      22744     945       1.0               11.0   
29      22745     984       1.0               11.0   
...       ...     ...       ...                ...   
64643   87359  156353       1.0               11.0   
64644   87360  156365       1.0               11.0   
64645   87361  156366       1.0               11.0   
64646   87362  156373       1.0               11.0   
64647   87363  156375       1.0               11.0   
64648   87364  156376       1.0               11.0   
64649   87365  156391       1.0               11.0   
64650   87366  156394       1.0               11.0   
64651   87367  156403       1.0               11.0   
64652   87368  156418       1.0               11.0   
64653   87369  156433       1.0               11.0   
64654   87370  156442       1.0               11.0   
64655   87371  156446       1.0               11.0   
64656   87372  156449       1.0               11.0   
64657   87373  156480       1.0               11.0   
64658   87374  156482       1.0               11.0   
64659   87375  156505       1.0               11.0   
64660   87376  156526       1.0               11.0   
64661   87377  156534       1.0               11.0   
64662   87378  156545       1.0               11.0   
64663   87379  156548       1.0               11.0   
64664   87380  156551       1.0               11.0   
64665   87381  156575       1.0               11.0   
64666   87382  156594       1.0               11.0   
64667   87383  156597       1.0               11.0   
64668   87384  156610       1.0               11.0   
64669   87385  156611       1.0               11.0   
64670   87386  156612       1.0               11.0   
64671   87387  156623       1.0               11.0   
64672   87388  156624       1.0               11.0   

                                             create_memo  task_source  \
0                                开始说借后面没就没上线了，现在话都不回一个，唉         22.0   
1      他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0   
2                               骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0   
3                      他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0   
4      他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0   
5                        这是交的主持会费，交完后我就后悔了叫她返会费，就直接拉黑我了。         22.0   
6                          他以玩游戏充值送分为由，骗了我五千元，结果人都联系不上了。         22.0   
7      盗用我朋友的QQ号像我骗取4000元钱，一次是通过微信扫描二维码转账给对方1200，一次是通...         22.0   
8

textid      id  is_fraud  finish_first_type  \
0       22716      15       1.0               11.0   
1       22717      30       1.0               11.0   
2       22718      43       1.0               11.0   
3       22719      56       1.0               11.0   
4       22720      60       1.0               11.0   
5       22721      64       1.0               11.0   
6       22722     119       1.0               11.0   
7       22723     121       1.0               11.0   
8       22724     158       1.0               11.0   
9       22725     179       1.0               11.0   
10      22726     222       1.0               11.0   
11      22727     303       1.0               11.0   
12      22728     325       1.0               11.0   
13      22729     346       1.0               11.0   
14      22730     406       1.0               11.0   
15      22731     422       1.0               11.0   
16      22732     435       1.0               11.0   
17      22733     470       1.0               11.0   
18      22734     479       1.0               11.0   
19      22735     490       1.0               11.0   
20      22736     520       1.0               11.0   
21      22737     542       1.0               11.0   
22      22738     578       1.0               11.0   
23      22739     668       1.0               11.0   
24      22740     765       1.0               11.0   
25      22741     826       1.0               11.0   
26      22742     831       1.0               11.0   
27      22743     832       1.0               11.0   
28      22744     945       1.0               11.0   
29      22745     984       1.0               11.0   
...       ...     ...       ...                ...   
64643   87359  156353       1.0               11.0   
64644   87360  156365       1.0               11.0   
64645   87361  156366       1.0               11.0   
64646   87362  156373       1.0               11.0   
64647   87363  156375       1.0               11.0   
64648   87364  156376       1.0               11.0   
64649   87365  156391       1.0               11.0   
64650   87366  156394       1.0               11.0   
64651   87367  156403       1.0               11.0   
64652   87368  156418       1.0               11.0   
64653   87369  156433       1.0               11.0   
64654   87370  156442       1.0               11.0   
64655   87371  156446       1.0               11.0   
64656   87372  156449       1.0               11.0   
64657   87373  156480       1.0               11.0   
64658   87374  156482       1.0               11.0   
64659   87375  156505       1.0               11.0   
64660   87376  156526       1.0               11.0   
64661   87377  156534       1.0               11.0   
64662   87378  156545       1.0               11.0   
64663   87379  156548       1.0               11.0   
64664   87380  156551       1.0               11.0   
64665   87381  156575       1.0               11.0   
64666   87382  156594       1.0               11.0   
64667   87383  156597       1.0               11.0   
64668   87384  156610       1.0               11.0   
64669   87385  156611       1.0               11.0   
64670   87386  156612       1.0               11.0   
64671   87387  156623       1.0               11.0   
64672   87388  156624       1.0               11.0   

                                             create_memo  task_source  \
0                                开始说借后面没就没上线了，现在话都不回一个，唉         22.0   
1      他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0   
2                               骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0   
3                      他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0   
4      他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0   
5                        这是交的主持会费，交完后我就后悔了叫她返会费，就直接拉黑我了。         22.0   
6                          他以玩游戏充值送分为由，骗了我五千元，结果人都联系不上了。         22.0   
7      盗用我朋友的QQ号像我骗取4000元钱，一次是通过微信扫描二维码转账给对方1200，一次是通...         22.0   
8

textid      id  is_fraud  finish_first_type  \
0       22716      15       1.0               11.0   
1       22717      30       1.0               11.0   
2       22718      43       1.0               11.0   
3       22719      56       1.0               11.0   
4       22720      60       1.0               11.0   
5       22721      64       1.0               11.0   
6       22722     119       1.0               11.0   
7       22723     121       1.0               11.0   
8       22724     158       1.0               11.0   
9       22725     179       1.0               11.0   
10      22726     222       1.0               11.0   
11      22727     303       1.0               11.0   
12      22728     325       1.0               11.0   
13      22729     346       1.0               11.0   
14      22730     406       1.0               11.0   
15      22731     422       1.0               11.0   
16      22732     435       1.0               11.0   
17      22733     470       1.0               11.0   
18      22734     479       1.0               11.0   
19      22735     490       1.0               11.0   
20      22736     520       1.0               11.0   
21      22737     542       1.0               11.0   
22      22738     578       1.0               11.0   
23      22739     668       1.0               11.0   
24      22740     765       1.0               11.0   
25      22741     826       1.0               11.0   
26      22742     831       1.0               11.0   
27      22743     832       1.0               11.0   
28      22744     945       1.0               11.0   
29      22745     984       1.0               11.0   
...       ...     ...       ...                ...   
64643   87359  156353       1.0               11.0   
64644   87360  156365       1.0               11.0   
64645   87361  156366       1.0               11.0   
64646   87362  156373       1.0               11.0   
64647   87363  156375       1.0               11.0   
64648   87364  156376       1.0               11.0   
64649   87365  156391       1.0               11.0   
64650   87366  156394       1.0               11.0   
64651   87367  156403       1.0               11.0   
64652   87368  156418       1.0               11.0   
64653   87369  156433       1.0               11.0   
64654   87370  156442       1.0               11.0   
64655   87371  156446       1.0               11.0   
64656   87372  156449       1.0               11.0   
64657   87373  156480       1.0               11.0   
64658   87374  156482       1.0               11.0   
64659   87375  156505       1.0               11.0   
64660   87376  156526       1.0               11.0   
64661   87377  156534       1.0               11.0   
64662   87378  156545       1.0               11.0   
64663   87379  156548       1.0               11.0   
64664   87380  156551       1.0               11.0   
64665   87381  156575       1.0               11.0   
64666   87382  156594       1.0               11.0   
64667   87383  156597       1.0               11.0   
64668   87384  156610       1.0               11.0   
64669   87385  156611       1.0               11.0   
64670   87386  156612       1.0               11.0   
64671   87387  156623       1.0               11.0   
64672   87388  156624       1.0               11.0   

                                             create_memo  task_source  \
0                                开始说借后面没就没上线了，现在话都不回一个，唉         22.0   
1      他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0   
2                               骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0   
3                      他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0   
4      他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0   
5                        这是交的主持会费，交完后我就后悔了叫她返会费，就直接拉黑我了。         22.0   
6                          他以玩游戏充值送分为由，骗了我五千元，结果人都联系不上了。         22.0   
7      盗用我朋友的QQ号像我骗取4000元钱，一次是通过微信扫描二维码转账给对方1200，一次是通...         22.0   
8

textid      id  is_fraud  finish_first_type  \
0       22716      15       1.0               11.0   
1       22717      30       1.0               11.0   
2       22718      43       1.0               11.0   
3       22719      56       1.0               11.0   
4       22720      60       1.0               11.0   
5       22721      64       1.0               11.0   
6       22722     119       1.0               11.0   
7       22723     121       1.0               11.0   
8       22724     158       1.0               11.0   
9       22725     179       1.0               11.0   
10      22726     222       1.0               11.0   
11      22727     303       1.0               11.0   
12      22728     325       1.0               11.0   
13      22729     346       1.0               11.0   
14      22730     406       1.0               11.0   
15      22731     422       1.0               11.0   
16      22732     435       1.0               11.0   
17      22733     470       1.0               11.0   
18      22734     479       1.0               11.0   
19      22735     490       1.0               11.0   
20      22736     520       1.0               11.0   
21      22737     542       1.0               11.0   
22      22738     578       1.0               11.0   
23      22739     668       1.0               11.0   
24      22740     765       1.0               11.0   
25      22741     826       1.0               11.0   
26      22742     831       1.0               11.0   
27      22743     832       1.0               11.0   
28      22744     945       1.0               11.0   
29      22745     984       1.0               11.0   
...       ...     ...       ...                ...   
64643   87359  156353       1.0               11.0   
64644   87360  156365       1.0               11.0   
64645   87361  156366       1.0               11.0   
64646   87362  156373       1.0               11.0   
64647   87363  156375       1.0               11.0   
64648   87364  156376       1.0               11.0   
64649   87365  156391       1.0               11.0   
64650   87366  156394       1.0               11.0   
64651   87367  156403       1.0               11.0   
64652   87368  156418       1.0               11.0   
64653   87369  156433       1.0               11.0   
64654   87370  156442       1.0               11.0   
64655   87371  156446       1.0               11.0   
64656   87372  156449       1.0               11.0   
64657   87373  156480       1.0               11.0   
64658   87374  156482       1.0               11.0   
64659   87375  156505       1.0               11.0   
64660   87376  156526       1.0               11.0   
64661   87377  156534       1.0               11.0   
64662   87378  156545       1.0               11.0   
64663   87379  156548       1.0               11.0   
64664   87380  156551       1.0               11.0   
64665   87381  156575       1.0               11.0   
64666   87382  156594       1.0               11.0   
64667   87383  156597       1.0               11.0   
64668   87384  156610       1.0               11.0   
64669   87385  156611       1.0               11.0   
64670   87386  156612       1.0               11.0   
64671   87387  156623       1.0               11.0   
64672   87388  156624       1.0               11.0   

                                             create_memo  task_source  \
0                                开始说借后面没就没上线了，现在话都不回一个，唉         22.0   
1      他诈骗说能开通备用金，你们可以去看一下好多诈骗，说可以开通备用金，叫我转账30给他，然后就把...         22.0   
2                               骗我交钱小说个小的然后一直要钱，不退钱，接单没钱         22.0   
3                      他以贷款流水不足 要求我转帐过去刷流水 最后说返钱给我 却把我拉黑         22.0   
4      他说给我单子让我刷，说先用一张没有钱的银行卡支付，然后会跳出余额不足，然后在用花呗支付，他说...         22.0   
5                        这是交的主持会费，交完后我就后悔了叫她返会费，就直接拉黑我了。         22.0   
6                          他以玩游戏充值送分为由，骗了我五千元，结果人都联系不上了。         22.0   
7      盗用我朋友的QQ号像我骗取4000元钱，一次是通过微信扫描二维码转账给对方1200，一次是通...         22.0   
8

In [56]:
before.to_csv('totaldata/fraudtext_unused.csv',encoding='utf-8',index=False)